# FS3 - MP analysis of all normally induced fates

In [ ]:
import os

dir = r"C:\Users\flori\OneDrive - Universität Wien\Adameyko Lab\FS3_MP\fluorescence_images"
os.chdir(dir)
dirs = os.listdir(".")
print(dirs)

In [ ]:
dirs = dirs[2:4]
print(dirs)

In [ ]:
from micropattern_analysis import *
import skimage.filters as filters
import skimage.exposure as exp
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
"""
    (common/custom
"""
images = {}
dapi_channel_number = 3
frames = []
"""
    common/custom)
"""

In [ ]:
im = iio.imread(next(gather_files(dirs))).max(axis=0)

colors = "jet"
fig, ax = plt.subplots(ncols=4, nrows=4, figsize=(15, 20))
for i in range(4):
    thresholds = filters.threshold_multiotsu(im[i], classes=3)
    regions = np.digitize(im[i], bins=thresholds)
    applied = (im[i] > thresholds[0]) * im[i]
    ax[i][0].imshow(im[i], cmap=colors)
    ax[i][0].set_title('Original')
    ax[i][0].axis('off')
    ax[i][1].imshow(regions, cmap=colors)
    ax[i][1].set_title('Triangle thresholding mask')
    ax[i][1].axis('off')
    ax[i][2].imshow(applied, cmap=colors)
    ax[i][2].set_title('Triangle thresholding applied')
    ax[i][2].axis('off')
    ax[i][3].hist(applied.ravel(), bins=50, color="blue")
plt.show()

In [ ]:
filters.try_all_threshold(im[1])

In [ ]:
plt.hist(im[3].ravel(), bins=200)
plt.show()

In [ ]:
"""
    (common
"""
for file in gather_files(dirs):
    img = iio.imread(file)
    img = maximise_img_channels(img, bits=1)

    images.update({file: img})

    dapi_img = img[dapi_channel_number]
    img_mask = create_img_mask(img, dapi_img, threshold_triangle)
    applied_img_mask = apply_img_mask(img, img_mask)
    new_applied_img_mask = applied_img_mask.copy()
    for channel_num in range(dapi_channel_number):
        thresholds = filters.threshold_multiotsu(applied_img_mask[channel_num], classes=3)
        new_applied_img_mask[channel_num] = (applied_img_mask[channel_num] > thresholds[1]) * applied_img_mask[channel_num]
    center_of_mass = get_center_of_mass(img_mask, dapi_img)
    cords = expand_coordinate_matrix(dapi_img)
    
    dist = calculate_distances(cords, center_of_mass)
    """
        common)
    """
    """
        (custom
    """
    file_params = os.path.basename(file).split("_")
    channel_names = file_params[6:12][::2]
    channel_names = list(map(lambda x: x.upper(), channel_names))
    channel_names.append("DAPI")
    mp_type = file_params[4:5]
    b = np.repeat(mp_type, len(dist))
    """
        custom)
    """
    """
        (common
    """
    a = np.repeat(center_of_mass, len(dist))
    """
        common)
    """
    """
        (custom
    """
    if "PAX6" in channel_names:
        """
            (common parts
        """
        df_mini = generate_data_frame(cords,
                                  applied_img_mask, 
                                  channel_names,
                                  Distances=dist,
                                  Center_of_Mass_x=np.repeat(center_of_mass[0], len(dist)),
                                  Center_of_Mass_y=np.repeat(center_of_mass[1], len(dist)),
                                  PAX6new=applied_img_mask[0].ravel(),
                                  SOX10new=applied_img_mask[1].ravel(),
                                  ISI12new=applied_img_mask[2].ravel())
        """
            common parts)
        """
    else:
        df_mini = generate_data_frame(cords,
                                  applied_img_mask, 
                                  channel_names,
                                  Distances=dist,
                                  Center_of_Mass_x=np.repeat(center_of_mass[0], len(dist)),
                                  Center_of_Mass_y=np.repeat(center_of_mass[1], len(dist)),
                                  SIX1new=applied_img_mask[0].ravel(),
                                  SOX10new=applied_img_mask[1].ravel(),
                                  ISI12new=applied_img_mask[2].ravel())
    """
        custom)
    """
    """
        (common/custom
    """
    img_props = iio.improps(file)
    img_meta = iio.immeta(file)
    if img_props.spacing:
        mean_res = np.mean(1 / np.array(img_props.spacing))
    elif img_meta["ScanInformation"]["SampleSpacing"]:
        mean_res = img_meta["ScanInformation"]["SampleSpacing"]
    else:
        mean_res = 1

    df_mini = scale_distances(df_mini, mean_res)
    """"
        common/custom)
    """
    """
        (custom
    """
    df_mini = group_distances(df_mini, channel_names)
    
    #df_mini = smooth_distances(df_mini, channel_names, sigma=5)
    df_mini["MP Type"] = np.repeat(mp_type, df_mini.shape[0])
    """
        custom)
    """
    frames.append(df_mini)

In [ ]:
"""
    (common
"""
df = pd.concat(frames)
"""
    common)
"""

In [ ]:
fig, (a, b) = plt.subplots(2, 1, figsize=(10, 10))

densities = df[(df["Distances"] < 400)].filter(("PAX6", "ISI12", "SOX10", "SIX1", "MP Type"))
densities.set_index("MP Type")
densities = densities.reset_index()
densities = pd.melt(densities, id_vars="MP Type", value_vars=["PAX6", "ISI12", "SOX10", "SIX1"])
sns.violinplot(densities[densities["value"] > 0], y="value", x="variable", hue="MP Type", ax=a)

densities = df[(df["Distances"] < 400)].filter(("PAX6new", "ISI12new", "SOX10new", "SIX1new", "MP Type"))
densities.set_index("MP Type")
densities = densities.reset_index()
densities = pd.melt(densities, id_vars="MP Type", value_vars=["PAX6new", "ISI12new", "SOX10new", "SIX1new"])
sns.violinplot(densities[densities["value"] > 0], y="value", x="variable", hue="MP Type", ax=b)

In [ ]:
import matplotlib.ticker as ticker


def plot_mp_types(pattern_size_order=("800um", "900um", "stencil"),
                  staining_order=("PAX6", "SIX1", "SOX10", "ISI12")):
    ticks = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(round(x * mean_res)))
    staining_order = list(map(str.lower, staining_order))
    img_map = {
        "pax6": 0,
        "six1": 0,
        "sox10": 1,
        "isi12": 2
    }
    pos_map = {
        "800um": 800,
        "900um": 900,
        "stencil": 900
    }

    (fig, axes) = plt.subplots(len(channel_names) * 2, 3, figsize=(35, 20))
    for i, so in zip(range(len(axes)), list(np.repeat(staining_order, 2))):
        if i % 2 == 0:
            cur_so = [f for f in images.keys() if so in f]
            for axi, pos in zip(axes[i], pattern_size_order):
                cur_pos = [f for f in cur_so if pos in f]
                key = np.random.choice(cur_pos)
                img = images.get(key)[img_map.get(so)]
                im = axi.imshow(img, cmap="jet")
                #axi.set_ylabel(channel_names[i])
                #ax.xaxis.set_major_formatter(ticks)
                #ax.yaxis.set_major_formatter(ticks)
                #pos = fig.add_axes([0.93, 0.1, 0.02, 0.35])
                #fig.colorbar(im, cax=pos)
                axi.set_title(pos) if i == 0 else None
        else:
            for axi, pos in zip(axes[i], pattern_size_order):
                df_sub = df[df["MP Type"] == pos]
                df_sub.reset_index(inplace=True)
                g = sns.lineplot(df_sub, x="Distances", y=f'{so.upper()}new', ax=axi)
                axi.set_xlim(0, pos_map.get(pos)/2)
                #axi.set_ylim(0, 1)
                axi.set_ylabel("Average Intensity [au]") if pos == pattern_size_order[0] else None
                axi.set_xlabel("")
                pos = axi.get_position()
                axi.text(pos.x0-0.25, pos.y0+0.25, so.upper()) if pos == pattern_size_order[0] else None
    fig.supxlabel("Distance [µm]")
    plt.grid(False)
    plt.subplots_adjust(left=0.25)
    plt.show()

In [ ]:
plot_mp_types()

In [ ]:
df[df["MP Type"] == "800um"].groupby("Distances").count()

In [ ]:
from matplotlib.patches import Circle
from matplotlib.patches import Rectangle

fig, ax = plt.subplots(6, 3, figsize=(7, 10))
keys = list(images.keys())[:9]
bar_width = 200
bar_padding = 50

for i in range(6):
    if i%2 == 0:
        for j in range(3):
            im = images.get(keys[i+j])[0]
            heatmap = ax[i][j].imshow(im, cmap="jet")
            fig.colorbar(heatmap, ax=ax[i][j], location="left") if j == 0 else None
            circ = Circle((500,500), 400, fill=False, linewidth=0.7, edgecolor="white", linestyle="dashed")
            rec = Rectangle((im.shape[0] - (bar_width + bar_padding), im.shape[]))
            ax[i][j].add_patch(circ)
            ax[i][j].axis("off")
    else:
        for j in range(3):
            ax[i][j].plot(np.linspace(0, 1), np.sin(np.linspace(0,1)))
            asp = np.diff(ax[i][j].get_xlim())[0] / np.diff(ax[i][j].get_ylim())[0]
            ax[i][j].set_aspect(asp)
plt.show()